In [ ]:
#Descarga del diccionario (ingles-chino) CC-CEDICT desde https://www.mdbg.net/chinese/dictionary?page=cc-cedict
# Descargamos el archivo CC-CEDICT desde el sitio oficial de MDBG
!wget https://www.mdbg.net/chinese/export/cedict/cedict_1_0_ts_utf-8_mdbg.txt -O cedict.txt

In [ ]:
#Una vez descargado el diccionario lo importamos ( se puede encontrar el archivo en mis documentos)
from google.colab import files
uploaded = files.upload()

In [ ]:
import gzip
import shutil

# Descomprime el .gz y lo guarda como .txt para poder procesar
with gzip.open("cedict_1_0_ts_utf-8_mdbg.txt.gz", 'rb') as f_in:
    with open("cedict.txt", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
#el objetivo es extraer del diccionario las palabras chinas ambiguas,es decir, aquellas que tienen más de un significado, y opcionalmente traducirlas al español.


In [ ]:
#leemos y procesamos el archivo
from collections import defaultdict

# Creamos un diccionario donde cada palabra tendrá una lista de significados
diccionario = defaultdict(list)

# Abrimos el archivo .txt ya descomprimido
with open("cedict.txt", encoding="utf-8") as f:
    for linea in f:
        if linea.startswith("#"):
            continue  # ignoramos las líneas de comentarios
        try:
            partes = linea.strip().split(" ")
            palabra = partes[0]  # lo primero que aparece es la palabra china
            significados = linea.split("/")  # los significados están entre barras "/"
            if len(significados) > 2:
                diccionario[palabra] = significados[1:-1]  # eliminamos los bordes vacíos
        except:
            continue  # si alguna línea da error, la ignoramos

In [ ]:
#Filtramos solo las palabras que tienen más de un significado
palabras_ambiguas = {k: v for k, v in diccionario.items() if len(v) > 1}

print("Palabras ambiguas encontradas:", len(palabras_ambiguas))

In [ ]:
#vamos a ver un diccionario, donde cada clave es una palabra o sigla en chino (o combinación de letras),
#y su valor es una lista de significados diferentes. Es decir, estamos viendo palabras ambiguas con múltiples sentidos.
# Mostramos algunas palabras ambiguas para revisar
for palabra, significados in list(palabras_ambiguas.items())[:10]:
    print(f"{palabra}: {significados}")

In [ ]:
# Ordenar palabras ambiguas por número de significados
mas_ambiguas = sorted(palabras_ambiguas.items(), key=lambda x: len(x[1]), reverse=True)

# Mostrar las más ambiguas
for palabra, significados in mas_ambiguas[:300]:
    print(f"{palabra} ({len(significados)} significados): {significados}")

In [ ]:
#ahora el objetivo es procesar las palabras ambiguas para identificar cuáles son las más importantes o buscadas,
#habra varias formas de hacerlo dependiendo de qué consideremos como “importantes”, como las mas frecuentes en el lenguaje chino, para ello cargamos un archivo
#de palabras frecuentes del chino para luego cruzarlas con las del otro diccionario


In [ ]:
#Cargamos un archivo de palabras frecuentes del chino de una base de datos publica
# Descargar el archivo dict.txt directamente desde GitHub
!wget https://github.com/fxsjy/jieba/raw/master/jieba/dict.txt -O dict.txt


In [ ]:
#Creamos un diccionario de frecuencias que usaremos para cruzarlas con las palabras del diccionario
#leemos y procesamos el archivo
frecuencias = {}#guardara cada palabra como clave y su frecuencia como valor
with open("dict.txt", encoding="utf-8") as f:
    for linea in f: #recorre el archivo por lineas
        partes = linea.strip().split()#limpiamos espacios en blanco
        if len(partes) >= 2:#aseguramos que al menos tenga la palabra y su frecuencia
            palabra = partes[0]
            frecuencia = int(partes[1])
            frecuencias[palabra] = frecuencia

In [ ]:
#construimos un ranking de las palabras chinas ambiguas mas importantes, combinando: numero de significados( mas de 3) y que tan frecuentes son y cortas y comunes.
ranking = []
for palabra in palabras_ambiguas:
   if palabra in frecuencias:
        peso = len(palabras_ambiguas[palabra]) * frecuencias[palabra]
        ranking.append((palabra, peso, palabras_ambiguas[palabra]))

ranking.sort(key=lambda x: x[1], reverse=True)

for p in ranking[:20]:
    print(f"{p[0]} (score {p[1]}): {p[2]}")

In [ ]:
#cruzamos los dos diccionarios para ver que palabras son mas frecuentes y mas relevantes y calculamos una puntuacion
ranking = []

for palabra in palabras_ambiguas:
    if palabra in frecuencias:
        # Score = número de significados * frecuencia de uso
        score = len(palabras_ambiguas[palabra]) * frecuencias[palabra]
        ranking.append((palabra, score, palabras_ambiguas[palabra]))


In [ ]:
#ordenamos de mayor a menor
ranking.sort(key=lambda x: x[1], reverse=True)

In [ ]:
#vemos el ranking de 20 palabras
for palabra, score, significados in ranking[:20]:
    print(f"{palabra} (score: {score}) → {significados}")


In [ ]:
print("Total de palabras ambiguas:", len(palabras_ambiguas))

In [ ]:
#palabras que tengan 3 o mas significados
ranking_muy_ambiguas = [p for p in ranking if len(p[2]) > 3]
print("Palabras con > 3 significados:", len(ranking_muy_ambiguas))

In [ ]:
#usar stopwords para quitar palabras muy basicas, aunque ambiguas son gramaticales y no interesantes......
stopwords = ["是", "有", "在", "的", "了", "和", "我", "你", "他"]
ranking_utiles = [p for p in ranking if p[0] not in stopwords]
print("Palabras útiles después de quitar stopwords:", len(ranking_utiles))


In [ ]:
#damos mas peso a las palabras mas ambiguas que a las mas frecuentes
#objetivo de este codigo es construir un ranking de las palabras chinas mas ambiguas, combinando
#combinando cuantos significados tienen(ambiguedad)y que de frecuentes son en el idioma chino
ranking = [] #lista vacia para crear palabras con su puntuacion, (score)

for palabra in palabras_ambiguas:#recorre cada palabra que detecta ambigua
    if palabra in frecuencias: # solo se procesa si aparece en el diccionario de frecuencia
        significados = palabras_ambiguas[palabra]
        frecuencia = frecuencias[palabra]
        score = (len(significados) ** 2) * frecuencia
        ranking.append((palabra, score, significados))

# Ordenar
ranking.sort(key=lambda x: x[1], reverse=True)

# Mostrar resultados
for palabra, score, significados in ranking[:20]:
    print(f"{palabra} (score: {score}) → {significados}")

In [ ]:
# Mostrar el score mínimo encontrado
score_min = min([score for _, score, _ in ranking])
print("Score mínimo:", score_min)

ranking.sort(key=lambda x: x[1])  # ordenar de menor a mayor
print(f"Palabra con score mínimo: {ranking[0]}")


In [ ]:
#traduccion en pinyin
!pip install pypinyin

from pypinyin import lazy_pinyin

def obtener_pinyin(palabra):
    return " ".join(lazy_pinyin(palabra))

In [ ]:
#importo desde libreria el traductor que usa google para traducir al español
!pip install deep-translator

In [ ]:
from deep_translator import GoogleTranslator

traducido = GoogleTranslator(source='en', target='es').translate("to hit; to play")
print(traducido)

In [ ]:
#funcion para traducir del en-es cualquier text

def traducir_es(texto_en):
    try:
        return GoogleTranslator(source='en', target='es').translate(texto_en)
    except:
        return ""


In [ ]:
from google.colab import files
files.download("/content/top_300_ambiguous_words.csv")


In [ ]:
import pandas as pd

df = pd.read_csv("/content/top_300_ambiguas_completo.csv")

print(df.head(10))

In [ ]:
df.info()

In [ ]:

def obtener_pinyin(palabra):
    return " ".join(lazy_pinyin(palabra))

def traducir_es(texto_en):
    try:
        return GoogleTranslator(source='en', target='es').translate(texto_en)
    except:
        return ""

def audio_url_google(palabra):
    return f"https://translate.google.com/translate_tts?ie=UTF-8&q={palabra}&tl=zh&client=tw-ob"

#top_300 usando 'ranking' variable
top_300 = ranking[:300]


In [ ]:
top_20 = df.sort_values("Score", ascending=False).head(20)

for i, row in top_20.iterrows():
    print(f"{row['Palabra']} ({row['Pinyin']}): {row['Significados_es']}")

In [ ]:
#EMBEDDING DE LAS 20 PALABRAS MAS INTERESANTES

In [ ]:
!pip install transformers

In [ ]:
#obtenemos los embeddings con BERT
from transformers import BertTokenizer, BertModel
import torch

# Cargamos BERT en chino
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
model = BertModel.from_pretrained("bert-base-chinese")

# Lista de palabras ambiguas
palabras = ["打", "花", "看", "吃", "走", "开", "生", "上", "下", "起"]

# Función para obtener embeddings
def obtener_embedding(palabra):
    inputs = tokenizer(palabra, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

# Lista de palabras desde top_20
palabras = top_20["Palabra"].tolist()
vectores = [obtener_embedding(p) for p in palabras]

In [ ]:
#encontrar la palabra con mas significados
#nos aseguramos de que los significados están separados por punto y coma
df["Num_Significados"] = df["Significados_en"].apply(lambda x: len(x.split(";")))

# Seleccionar la fila con más significados
palabra_max = df.sort_values("Num_Significados", ascending=False).iloc[0]
print(f"Palabra con más significados: {palabra_max['Palabra']} ({palabra_max['Pinyin']})")
print("Significados:", palabra_max["Significados_es"])
# cuantos signficados tiene
print("Número de significados:", palabra_max["Num_Significados"])



In [ ]:
palabra = "上"
fila = df[df["Palabra"] == palabra].iloc[0]
significados = [s.strip() for s in fila["Significados_en"].split(";")]
print(f"Significados de {palabra}:")
for sig in significados:
    print("-", sig)



In [ ]:
# Embedding de la palabra en chino
embedding_palabra = obtener_embedding(palabra)

# Embeddings de los significados (en inglés)
embeddings_significados = [obtener_embedding(sig) for sig in significados]

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Unimos el embedding de la palabra con los de sus significados
todos_vectores = [embedding_palabra] + embeddings_significados
coords = PCA(n_components=2).fit_transform(todos_vectores)

# Gráfico
plt.figure(figsize=(12, 10))
plt.title(f"📍 Embeddings de los Significados de: {palabra} ({fila['Pinyin']})", fontsize=16)
plt.grid(True)

# Punto central: la palabra en rojo
plt.scatter(coords[0, 0], coords[0, 1], color='red', s=300, label="Palabra")
plt.text(coords[0, 0] + 0.02, coords[0, 1] + 0.02, f"{palabra} ({fila['Pinyin']})", fontsize=14, fontweight="bold")

# Puntos satélite: significados
for i, significado in enumerate(significados):
    x, y = coords[i + 1]
    plt.scatter(x, y, color='blue', s=150)
    plt.text(x + 0.01, y + 0.01, significado, fontsize=10)

plt.legend(["Palabra", "Significados"], loc="best")
plt.tight_layout()
plt.show()

In [ ]:
#clasificamos los significados por categoria
# Significados separados por punto y coma en tu CSV
significados = [
    "(bound form) up",
    "upper",
    "above",
    "previous",
    "first (of multiple parts)",
    "to climb",
    "to get onto",
    "to go up",
    "to attend (class or university)",
    "(directional complement) up",
    "(noun suffix) on",
    "above"
]

# Clasificación manual por categoría
categorias = [
    "dirección",       # up
    "posición",        # upper
    "posición",        # above
    "tiempo",          # previous
    "tiempo",          # first of multiple parts
    "acción",          # to climb
    "acción",          # to get onto
    "acción",          # to go up
    "actividad",       # to attend
    "dirección",       # complement up
    "gramática",       # noun suffix on
    "posición"         # above (de nuevo)
]

In [ ]:
#asignamos colores por categoria
# Asignamos un color a cada categoría
colores_categoria = {
    "dirección": "blue",
    "posición": "green",
    "tiempo": "orange",
    "acción": "purple",
    "actividad": "teal",
    "gramática": "brown"
}

In [ ]:
#visualizar por colores en grupo
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Calculamos coordenadas PCA
todos_vectores = [obtener_embedding(palabra)] + [obtener_embedding(s) for s in significados]
coords = PCA(n_components=2).fit_transform(todos_vectores)

# Gráfico
plt.figure(figsize=(12, 10))
plt.title(f"📍 Embeddings de los Significados de: {palabra} ({fila['Pinyin']})", fontsize=16)
plt.grid(True)

# Punto central (la palabra china)
plt.scatter(coords[0, 0], coords[0, 1], color='red', s=300, label="Palabra")
plt.text(coords[0, 0] + 0.02, coords[0, 1] + 0.02, f"{palabra} ({fila['Pinyin']})", fontsize=14, fontweight="bold")

# Puntos de los significados, coloreados
for i, significado in enumerate(significados):
    categoria = categorias[i]
    color = colores_categoria[categoria]
    x, y = coords[i + 1]
    plt.scatter(x, y, color=color, s=180, label=categoria if categoria not in plt.gca().get_legend_handles_labels()[1] else None)
    plt.text(x + 0.01, y + 0.01, significado, fontsize=10)

plt.legend(title="Categorías", loc="upper right")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import re

# Lista para guardar cada fila como un dict
datos = []

with open("cedict.txt", encoding="utf-8") as f:
    for linea in f:
        if linea.startswith("#"):
            continue
        try:
            # Línea típica: 中國 中国 [Zhong1 guo2] /China/medio país/
            trad = re.match(r"(\S+)\s+(\S+)\s+\[(.+?)\]\s+/(.+)/", linea)
            if trad:
                palabra_simp = trad.group(1)
                palabra_trad = trad.group(2)
                pinyin = trad.group(3)
                significados = trad.group(4).split('/')

                if len(significados) > 1:
                    for significado in significados:
                        datos.append({
                            "palabra_china": palabra_simp,
                            "pinyin": pinyin,
                            "significado": significado.strip(),
                            "categoria": "",
                            "uso_tipico": ""
                        })
        except:
            continue

# Creamos el DataFrame con los significados separados
df = pd.DataFrame(datos)
df.head(100)
